# Autocombat TOP and StrokeMRI vs. SABRE harmonized datasets

Note this must be run in the `comscan6` environment

## import libraries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

sys.path.insert(0, '../../') # path to cvasl functions

import cvasl.harmony as har
import cvasl.seperated as sep
import cvasl.vendor.comscan.neurocombat as autocombat

## Let's see how we would apply this to our data
We will have to flip it on it's side to make it work

In [ ]:
# Datasets for this work
# Datasets for this work
MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')
SABRE = pd.read_csv('../new_data/TrainingDataComplete_SABRE.csv')

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
SABRE = SABRE.drop(['ID'],axis = 1)
SABRE.tail(5)

In [ ]:
TOPMRI = pd.concat([TOP, MRI])
TOPMRI['Site'] = 0
TOPMRI.head(3) 

In [ ]:
datasets = [TOPMRI, SABRE]
new_frames = sep.deal_with_readout_and_labelling(datasets, ['M0'])

In [ ]:
# make sure things are inproper order
for frame in new_frames:
    print(len(frame))
print(len(TOPMRI))
print(len(SABRE))

In [ ]:
#new_frames[0]
TOPMRI = new_frames[0]
SABRE = new_frames[1]


In [ ]:
new_frame_datasets=  [TOPMRI, SABRE, ]
for everyone in new_frame_datasets:
    everyone.columns =  everyone.columns.str.lower()
TOPMRI.head(3)

In [ ]:
TOPMRI.columns = TOPMRI.columns.str.lower()

In [ ]:
SABRE.columns = SABRE.columns.str.lower()

In [ ]:
TOPMRISABRE = pd.concat([TOPMRI, SABRE])
TOPMRISABRE

In [ ]:
TOPMRISABRE['decade']=(TOPMRISABRE['age']/10).round()
#TOPMRI['decade']

In [ ]:
TOPMRISABRE = TOPMRISABRE.sort_values(by='age')
TOPMRISABRE.reset_index(inplace=True)
TOPMRISABRE['fine_grain'] = TOPMRISABRE['age'].rolling(2).sum()/2
#TOPMRISABEW

In [ ]:
TOPMRISABRE[2:].fine_grain.iloc[::2] = TOPMRISABRE[:].fine_grain.iloc[1::2]
#TOPMRI['fine_grain']

In [ ]:
TOPMRISABRE['fine_grain'][0] = TOPMRISABRE['fine_grain'][1]
TOPMRISABRE['fine_grain']

In [ ]:
TOPMRISABRE['decade'].unique()

In [ ]:
TOPMRISABRE.columns

In [ ]:
combat = autocombat.Combat(
    features=[# 'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
        'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
        'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
             #'ld', 'pld', 'labelling', 'readout',],
    sites=["site"], discrete_covariates=['sex'],continuous_covariates=['decade'],)

fg_combat = autocombat.Combat(
    features=[ #'gm_vol', 'wm_vol', 'csf_vol',
       #'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',],
            # 'ld', 'pld', 'labelling', 'readout',],
    sites=[ "site"], discrete_covariates=['sex'],continuous_covariates=['fine_grain'],)


In [ ]:
print(combat.fit(TOPMRISABRE))
print(fg_combat.fit(TOPMRISABRE))

In [ ]:
print(combat.gamma_star_)
print(fg_combat.gamma_star_)

In [ ]:
transformed_TOPMRISABRE= combat.transform(TOPMRISABRE)
transformed_TOPMRISABRE.head(3)

In [ ]:
fg_transformed_TOPMRISABRE= fg_combat.transform(TOPMRISABRE)
fg_transformed_TOPMRISABRE.head(3)

In [ ]:
# check what harmonization did
for column in ['age', 'sex', 'site', 'gm_vol', 'wm_vol',
       'csf_vol', 'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',
       'ld', 'pld', 'labelling', 'readout', 'decade', 'fine_grain']:
        
    print(column, sum(TOPMRISABRE[column] - transformed_TOPMRISABRE[column]))

In [ ]:
TOPMRISABRE.head(3)

In [ ]:
transformed_TOPMRISABRE= combat.transform(TOPMRISABRE)
transformed_TOPMRISABRE.head(3)

In [ ]:
fg_transformed_TOPMRISABRE= fg_combat.transform(TOPMRISABRE)
fg_transformed_TOPMRISABRE.head(3)

In [ ]:
transformed_TOPMRISABRE.columns

In [ ]:
TOPMRI_transformed = transformed_TOPMRISABRE[ transformed_TOPMRISABRE['site']==0]
TOPMRI_transformed = TOPMRI_transformed.drop(['site', 'decade','fine_grain', 'index'], axis=1)
TOPMRI_transformed.head(3)

In [ ]:
fg_TOPMRI_transformed = fg_transformed_TOPMRISABRE[ fg_transformed_TOPMRISABRE['site']==0]
fg_TOPMRI_transformed = fg_TOPMRI_transformed .drop(['site', 'decade','fine_grain', 'index'], axis=1)
fg_TOPMRI_transformed.head(3)

In [ ]:
SABRE_transformed = transformed_TOPMRISABRE[ transformed_TOPMRISABRE['site']==1]
SABRE_transformed = SABRE_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
SABRE_transformed.head(3)

In [ ]:
fg_SABRE_transformed = fg_transformed_TOPMRISABRE[fg_transformed_TOPMRISABRE['site']==1]
fg_SABRE_transformed = fg_SABRE_transformed.drop(['site', 'decade', 'index', 'fine_grain'], axis=1)
fg_SABRE_transformed.head(3)

In [ ]:
TOPMRI_transformed.to_csv('harm_results/autocombat/autocom_harm_topmri_v_s.csv')
SABRE_transformed.to_csv('harm_results/autocombat/autocom_harm_sabre.csv')

fg_TOPMRI_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_topmri_v_s.csv')
fg_SABRE_transformed.to_csv('harm_results/autocombat/fg_autocom_harm_sabre1.csv')

# Data quality check

In [ ]:
TOPMRI_transformed.isna().sum().sum()

In [ ]:
SABRE_transformed.isna().sum().sum()

In [ ]:
number_columns = ['sex', 'gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf',]

In [ ]:
#neuro_harm_top.head(3)
(TOPMRI_transformed[number_columns] < 0).sum()

In [ ]:
(SABRE_transformed[number_columns] < 0).sum()

In [ ]:
SABRE_transformed

## So some variables are again problematic, becuas they are negative

 We will wait about logging it until we see other columns we may want to log across all datasets